In [1]:
from gurobipy import *
from math import *

In [2]:


q=((0,0),(0,40),(0,50),(1,49),(46,50),(48,48),(46,46),(50,48),(50,35),(30,5),(30,0)
   ,(50,50),(34,50),(34,9),(0,13),(25,25),(50,34),(34,22),(53,9),(30,13),(15,25)
  ,(22,50),(4,50),(5,9),(6,13),(4,25),(8,34),(8,22),(32,9),(8,13),(30,25));

V_h=90
V_c=18


qs=(0,0)
qf=(50,0)

In [3]:
qf[0]

50

In [4]:
# Create a new model
m = Model("MISOCP_2014")
n = 30

# Create variables
Tx = {};
Ty = {};
lx = {};
ly = {};
alpha = {};
f = {};
s = {};


#for SOCP
Cx= {};
Cy= {};
C= {};

d= {};

for i in range(1, n+1):
    Tx[i] = m.addVar(vtype = 'C',name='Tx[%s]'%i);
    Ty[i] = m.addVar(vtype = 'C',name='Ty[%s]'%i);
    lx[i] = m.addVar(vtype = 'C',name='lx[%s]'%i);
    ly[i] = m.addVar(vtype = 'C',name='ly[%s]'%i);
    for j in range(i, n+1):
        alpha[i,j] = m.addVar(lb=0,vtype = 'B',name='alpha[%s,%s]'%(i,j));
        f[i,j] = m.addVar(lb=0,vtype = 'C',name='f[%s,%s]'%(i,j));
        s[i,j] = m.addVar(lb=0,vtype = 'C',name='s[%s,%s]'%(i,j));
        d[i,j]=quicksum(sqrt( (q[k][0]-q[k+1][0])*(q[k][0]-q[k+1][0]) + (q[k][1]-q[k+1][1])*(q[k][1]-q[k+1][1]) ) for k in range(i,j));
        


            


        
# Integrate new variables

m.update()




Academic license - for non-commercial use only


In [5]:

# Add constraint: 
M =200
ctr=0;

for i in range(1, n+1):
    for j in range(i,n+1):
        
        #C18
        m.addConstr( f[i,j] - 0.35 <= M*(1-alpha[i,j])  ,'C.18.[%s,%s]'%(i,j));
        
        
        
        #C19a
        
        
        ctr=ctr+1;
    
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19a.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19a.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C19a.[%s,%s]'%(i,j));
    
    
        m.addConstr( Cx[ctr]  == Tx[i]-lx[j], 'C19ax.1.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr]  == Ty[i]-ly[j], 'C19ay.1.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C19a.2.[%s,%s]'%(i,j));
        m.addConstr( C[ctr]-(V_c*f[i,j])  == M*(1-alpha[i,j]), 'C19a.3.[%s,%s]'%(i,j));
        
    
        
        #C19b
        
        ctr=ctr+1;
        
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19b.1.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19b.1.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C19b.1.[%s,%s]'%(i,j));
        
        ctr=ctr+1;
        
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19b.2.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19b.2.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C19b.2.[%s,%s]'%(i,j));
        
        
        m.update()
        m.addConstr( Cx[ctr-1]  == Tx[i]-q[i][0], 'C19bx.1.1.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr-1]  == Ty[i]-q[i][1], 'C19by.1.1.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr-1]*Cx[ctr-1] + Cy[ctr-1]*Cy[ctr-1] <=  C[ctr-1]*C[ctr-1], 'C19b.2.1.[%s,%s]'%(i,j));
        
        
        m.addConstr( Cx[ctr]  == lx[j]-q[j][0], 'C19bx.1.2.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr]  == ly[j]-q[j][1], 'C19by.1.2.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C19b.2.2.[%s,%s]'%(i,j));
        
       

       
        
        m.addConstr( C[ctr] + d[i,j]+C[ctr-1]-V_h*f[i,j] == M*(1-alpha[i,j]), 'C.19b.3.[%s,%s]'%(i,j));
        
        
     
        
        
for i in range(1, n+1):
    for j in range(i,n):

        #C19c
      
        
        ctr=ctr+1;
        
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19c.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19c.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C19c.[%s,%s]'%(i,j));
            
        m.addConstr( Cx[ctr]  == lx[j]-Tx[j+1], 'C19cx.1.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr]  == ly[j]-Ty[j+1], 'C19cy.1.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C19c.2.[%s,%s]'%(i,j));
        m.addConstr( C[ctr]-(V_c*s[i,j])  == M*(1-alpha[i,j]), 'C19c.3.[%s,%s]'%(i,j));
        
        

                 

#C2
for k in range(1, n+1):
    m.addConstr( quicksum(quicksum(alpha[i,j] for j in range(k,n+1)) for i in range(1,k+1))  == 1, 'C2.[%s]'%k); 
        
        


        



In [6]:
###Objective Function
a = m.addVar(lb=0,vtype = 'C',name='a');
b = m.addVar(lb=0,vtype = 'C',name='b');

ctr=ctr+1;
Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='nnn');


m.addConstr( Cx[ctr] == lx[n]-qf[0], 'obj_bx');



m.addQConstr( Tx[1]*Tx[1]+Ty[1]*Ty[1] <= a*a, 'obj_a');
m.addQConstr( Cx[ctr]*Cx[ctr] + ly[n]*ly[n] <= b*b, 'obj_b');


obj = (a+b)/V_c + quicksum(quicksum(f[i,j] for j in range(i,n+1)) for i in range(1,n+1) ) + quicksum(quicksum(s[i,j] for j in range(i,n)) for i in range(1,n+1)) 
m.setObjective(obj,GRB.MINIMIZE)

In [7]:
m.optimize()

Optimize a model with 5521 rows, 7008 columns and 19572 nonzeros
Model has 1832 quadratic constraints
Variable types: 6543 continuous, 465 integer (465 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [6e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-02, 4e+02]
Presolve removed 3776 rows and 3866 columns
Presolve time: 0.23s
Presolved: 1745 rows, 3142 columns, 5124 nonzeros
Variable types: 3086 continuous, 56 integer (56 binary)
Presolve time: 0.26s
Presolved: 8803 rows, 5615 columns, 14655 nonzeros
Presolved model has 1832 second-order cone constraints
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 22
 AA' NZ     : 1.521e+05
 Factor NZ  : 8.095e+05 (roughly 12 MBytes of memory)
 Factor Ops : 3.280e+08 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Com

  1969   574  427.15391   16    5  438.66672  423.97856  3.35%   0.0  609s
  2052   597     cutoff   13       438.66672  424.21297  3.29%   0.0  631s
  2136   608     cutoff   11       438.66672  424.24084  3.29%   0.0  654s
  2230   637  427.45106   12   41  438.66672  424.26383  3.28%   0.0  658s
  2234   640  425.30191   10   36  438.66672  424.26383  3.28%   0.0  660s
  2237   647  424.26383   13   34  438.66672  424.26383  3.28%   0.0  666s
  2245   652  424.26383   14   27  438.66672  424.26383  3.28%   0.0  670s
  2257   660  424.26383   16   28  438.66672  424.26383  3.28%   0.0  676s
  2265   659  424.26383   17   24  438.66672  424.26383  3.28%   0.0  680s
  2277   673  424.26383   18   15  438.66672  424.26383  3.28%   0.0  686s
  2290   675  425.67044   20    8  438.66672  424.26383  3.28%   0.0  691s
  2299   678  428.93987   21    4  438.66672  424.26383  3.28%   0.0  695s
  2320   686  434.53455   23    2  438.66672  424.26383  3.28%   0.0  701s
  2336   677 infeasible  

In [8]:

obj = m.getObjective()
print("t_m:",obj.getValue(),"hours\n")
print("M:",M,"\n")
ctr=0
for i in range(1,n+1):
    for j in range(i,n+1):
        if alpha[i,j].X >0.1:
            ctr=ctr+1
            print("Flyover no.", ctr, "from q",i, " to q",j,":\n" )
            print("T", i," : (",Tx[i].X,", ", Ty[i].X,")")
            print("l", j," : (",lx[j].X,", ", ly[j].X,")\n")

t_m: 438.6667193935566 hours

M: 200 

Flyover no. 1 from q 1  to q 3 :

T 1  : ( 8.39407879789401e-06 ,  39.999988971965195 )
l 3  : ( 5.103561263422738 ,  41.10941819171009 )

Flyover no. 2 from q 4  to q 5 :

T 4  : ( 45.99998125861146 ,  49.99998043863468 )
l 5  : ( 45.99986927992617 ,  47.06006834937267 )

Flyover no. 3 from q 6  to q 6 :

T 6  : ( 46.00000438476659 ,  46.000009435006724 )
l 6  : ( 48.016931434357794 ,  47.00838006843972 )

Flyover no. 4 from q 7  to q 8 :

T 7  : ( 49.99997156935968 ,  47.99997197019396 )
l 8  : ( 47.514646959973135 ,  42.6564665006556 )

Flyover no. 5 from q 9  to q 10 :

T 9  : ( 30.000007865326015 ,  5.000028302080313 )
l 10  : ( 31.773931619312634 ,  8.9913985560354 )

Flyover no. 6 from q 11  to q 12 :

T 11  : ( 49.999951843062966 ,  49.99995298700842 )
l 12  : ( 46.74466755099518 ,  44.60621757652374 )

Flyover no. 7 from q 13  to q 13 :

T 13  : ( 33.999985112782596 ,  9.000014459270542 )
l 13  : ( 33.99995751032413 ,  9.000009769122679 )

In [9]:
print(d[1,2])

<gurobi.LinExpr: 10.0>


In [10]:
m.getVars()

[<gurobi.Var Tx[1] (value 8.39407879789401e-06)>,
 <gurobi.Var Ty[1] (value 39.999988971965195)>,
 <gurobi.Var lx[1] (value 107.94599861943594)>,
 <gurobi.Var ly[1] (value 132.43823310615386)>,
 <gurobi.Var alpha[1,1] (value 0.0)>,
 <gurobi.Var f[1,1] (value 2.1405671013496885e-07)>,
 <gurobi.Var s[1,1] (value 9.537839673612325e-08)>,
 <gurobi.Var alpha[1,2] (value 0.0)>,
 <gurobi.Var f[1,2] (value 4.619522471216442e-07)>,
 <gurobi.Var s[1,2] (value 9.640508125130509e-08)>,
 <gurobi.Var alpha[1,3] (value 1.0)>,
 <gurobi.Var f[1,3] (value 0.29015274970384924)>,
 <gurobi.Var s[1,3] (value 2.3250909025567577)>,
 <gurobi.Var alpha[1,4] (value 0.0)>,
 <gurobi.Var f[1,4] (value 1.6339385489061442e-07)>,
 <gurobi.Var s[1,4] (value 9.92460353899105e-08)>,
 <gurobi.Var alpha[1,5] (value 0.0)>,
 <gurobi.Var f[1,5] (value 1.346759681553067e-07)>,
 <gurobi.Var s[1,5] (value 0.0)>,
 <gurobi.Var alpha[1,6] (value 0.0)>,
 <gurobi.Var f[1,6] (value 1.3479160365648113e-07)>,
 <gurobi.Var s[1,6] (value 

In [11]:
f

{(1, 1): <gurobi.Var f[1,1] (value 2.1405671013496885e-07)>,
 (1, 2): <gurobi.Var f[1,2] (value 4.619522471216442e-07)>,
 (1, 3): <gurobi.Var f[1,3] (value 0.29015274970384924)>,
 (1, 4): <gurobi.Var f[1,4] (value 1.6339385489061442e-07)>,
 (1, 5): <gurobi.Var f[1,5] (value 1.346759681553067e-07)>,
 (1, 6): <gurobi.Var f[1,6] (value 1.3479160365648113e-07)>,
 (1, 7): <gurobi.Var f[1,7] (value 1.6876972175339233e-07)>,
 (1, 8): <gurobi.Var f[1,8] (value 1.3569299461835271e-07)>,
 (1, 9): <gurobi.Var f[1,9] (value 1.8768880671871102e-07)>,
 (1, 10): <gurobi.Var f[1,10] (value 1.4622115251938794e-07)>,
 (1, 11): <gurobi.Var f[1,11] (value 2.0890078111790444e-07)>,
 (1, 12): <gurobi.Var f[1,12] (value 0.04778584844980571)>,
 (1, 13): <gurobi.Var f[1,13] (value 0.3495732616561291)>,
 (1, 14): <gurobi.Var f[1,14] (value 0.7299564469531497)>,
 (1, 15): <gurobi.Var f[1,15] (value 1.038076704662437)>,
 (1, 16): <gurobi.Var f[1,16] (value 1.3333066558947788)>,
 (1, 17): <gurobi.Var f[1,17] (valu

In [12]:
alpha

{(1, 1): <gurobi.Var alpha[1,1] (value 0.0)>,
 (1, 2): <gurobi.Var alpha[1,2] (value 0.0)>,
 (1, 3): <gurobi.Var alpha[1,3] (value 1.0)>,
 (1, 4): <gurobi.Var alpha[1,4] (value 0.0)>,
 (1, 5): <gurobi.Var alpha[1,5] (value 0.0)>,
 (1, 6): <gurobi.Var alpha[1,6] (value 0.0)>,
 (1, 7): <gurobi.Var alpha[1,7] (value 0.0)>,
 (1, 8): <gurobi.Var alpha[1,8] (value 0.0)>,
 (1, 9): <gurobi.Var alpha[1,9] (value 0.0)>,
 (1, 10): <gurobi.Var alpha[1,10] (value 0.0)>,
 (1, 11): <gurobi.Var alpha[1,11] (value 0.0)>,
 (1, 12): <gurobi.Var alpha[1,12] (value 0.0)>,
 (1, 13): <gurobi.Var alpha[1,13] (value 0.0)>,
 (1, 14): <gurobi.Var alpha[1,14] (value 0.0)>,
 (1, 15): <gurobi.Var alpha[1,15] (value 0.0)>,
 (1, 16): <gurobi.Var alpha[1,16] (value 0.0)>,
 (1, 17): <gurobi.Var alpha[1,17] (value 0.0)>,
 (1, 18): <gurobi.Var alpha[1,18] (value 0.0)>,
 (1, 19): <gurobi.Var alpha[1,19] (value 0.0)>,
 (1, 20): <gurobi.Var alpha[1,20] (value 0.0)>,
 (1, 21): <gurobi.Var alpha[1,21] (value 0.0)>,
 (1, 22): 

In [13]:
#m.write("out.lp")

In [14]:
#m.computeIIS()

In [15]:
status = m.status
if status == GRB.Status.OPTIMAL:
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
    print('Obj: %g' % m.objVal)
       
elif status == GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)
    # do IIS
    m.computeIIS()
    for c in m.getConstrs():
        if c.IISConstr:
            print('%s' % c.constrName)

In [16]:
#m.write("SOCP_2014.lp")

In [17]:
#m.computeIIS()

In [18]:
obj = m.getObjective()
print(obj.getValue())

438.6667193935566


In [19]:
M

200